In [89]:
import requests
from datetime import datetime, timezone
import pandas as pd
import numpy as np

time_window = 21600  # 21600 seconds = 6h
limit = 500 #  how many datapoints per request. Max 1000 but if you put it at 1000, it does not work
pairs = ['btcusd', 'ethusd']


start_date = datetime(2017,9,1,0,0,0,tzinfo=timezone.utc).timestamp()  # 2017-8-18 is the first date bitstamp has ETH/USD data, for some reason
end_date = datetime(2021,1,1,0,0,0,tzinfo=timezone.utc).timestamp()

time_span = (end_date - start_date)# total amount of seconds between the dates
print('Time span in seconds: ' + str(time_span))
total_candles = time_span/time_window  # each candle covers time_window seconds, so this is how many candles we have
total_requests_required = int(total_candles/limit) +1  # each request can contain max 1000 data points


cols = ['timestamp']
cols.extend(pairs)  # .append method adds pairs as a list within the cols list

df = pd.DataFrame(columns=cols)

for i,j in zip(pairs, df.columns[1:]):  # i is for fetching data, j is for inserting into df

    timestamp = np.array([])
    closing_price = np.array([])

    step_size = time_window * limit # can only get 1000 obs per request, so need to break it up
    start = start_date
    #print('start timestamp: ' + str(start) + ' which is ' + str(datetime.utcfromtimestamp(start)))
    print(i)
    for k in range(total_requests_required):
        print('From ' + str(datetime.utcfromtimestamp(start)) + ' to ' + str(datetime.utcfromtimestamp(start+step_size)))
        params = {'start': int(start), 'end': int(start+step_size), 'step': time_window, 'limit': limit}
        r = requests.get('https://www.bitstamp.net/api/v2/ohlc/' + i , params=params)
        r_dict = r.json()

        for q in range(len(r_dict['data']['ohlc'])):
            timestamp = np.append(timestamp, float(r_dict['data']['ohlc'][q]['timestamp']))
            closing_price = np.append(closing_price, float(r_dict['data']['ohlc'][q]['close']))

        start = start + step_size

    if i == 'btcusd':
        timestamp_copy = timestamp


    print(len(timestamp))
    print(len(closing_price))
    # df[j] = closing_price

Time span in seconds: 105235200.0
btcusd
From 2017-09-01 00:00:00 to 2018-01-04 00:00:00
From 2018-01-04 00:00:00 to 2018-05-09 00:00:00
From 2018-05-09 00:00:00 to 2018-09-11 00:00:00
From 2018-09-11 00:00:00 to 2019-01-14 00:00:00
From 2019-01-14 00:00:00 to 2019-05-19 00:00:00
From 2019-05-19 00:00:00 to 2019-09-21 00:00:00
From 2019-09-21 00:00:00 to 2020-01-24 00:00:00
From 2020-01-24 00:00:00 to 2020-05-28 00:00:00
From 2020-05-28 00:00:00 to 2020-09-30 00:00:00
From 2020-09-30 00:00:00 to 2021-02-02 00:00:00
4963
4963
ethusd
From 2017-09-01 00:00:00 to 2018-01-04 00:00:00
From 2018-01-04 00:00:00 to 2018-05-09 00:00:00
From 2018-05-09 00:00:00 to 2018-09-11 00:00:00
From 2018-09-11 00:00:00 to 2019-01-14 00:00:00
From 2019-01-14 00:00:00 to 2019-05-19 00:00:00
From 2019-05-19 00:00:00 to 2019-09-21 00:00:00
From 2019-09-21 00:00:00 to 2020-01-24 00:00:00
From 2020-01-24 00:00:00 to 2020-05-28 00:00:00
From 2020-05-28 00:00:00 to 2020-09-30 00:00:00
From 2020-09-30 00:00:00 to 20

In [72]:
print('First date in timestamp: ' + str(datetime.utcfromtimestamp(int(timestamp[0]))))
print('First date in timestamp_copy: ' + str(datetime.utcfromtimestamp(int(timestamp_copy[0]))))
print('Last date in timestamp: ' + str(datetime.utcfromtimestamp(int(timestamp[-1]))))
print('Last date in timestamp_copy: ' + str(datetime.utcfromtimestamp(int(timestamp_copy[-1]))))

First date in timestamp: 2017-10-02 06:00:00
First date in timestamp_copy: 2017-10-02 06:00:00
Last date in timestamp: 2019-02-14 00:00:00
Last date in timestamp_copy: 2019-02-14 00:00:00


In [85]:
diff = np.setdiff1d(timestamp, timestamp_copy)

In [86]:
diff[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [74]:
str(datetime.utcfromtimestamp(int(diff[0])))

'2018-06-09 06:00:00'

In [87]:
np.unique(np.diff(timestamp_copy))

array([21600.])

In [88]:
np.unique(np.diff(timestamp))

array([21600.])

In [82]:
params2 = {'start': int(diff[0]), 'end': int(diff[0]), 'step': time_window, 'limit': 1}
g = requests.get('https://www.bitstamp.net/api/v2/ohlc/btcusd', params=params2)
print(g.json())


{'data': {'pair': 'BTC/USD', 'ohlc': [{'high': '7686.45', 'timestamp': '1528524000', 'volume': '441.96510443', 'low': '7621.07', 'close': '7643.95', 'open': '7667.60'}]}}
